# Linear Regression Consulting Project

Congratulations! You've been contracted by Hyundai Heavy Industries to help them build a predictive model for some ships. [Hyundai Heavy Industries](http://www.hyundai.eu/en) is one of the world's largest ship manufacturing companies and builds cruise liners.

You've been flown to their headquarters in Ulsan, South Korea to help them give accurate estimates of how many crew members a ship will require.

They are currently building new ships for some customers and want you to create a model and use it to predict how many crew members the ships will need.

Here is what the data looks like so far:

    Description: Measurements of ship size, capacity, crew, and age for 158 cruise
    ships.


    Variables/Columns
    Ship Name     1-20
    Cruise Line   21-40
    Age (as of 2013)   46-48
    Tonnage (1000s of tons)   50-56
    passengers (100s)   58-64
    Length (100s of feet)  66-72
    Cabins  (100s)   74-80
    Passenger Density   82-88
    Crew  (100s)   90-96
    
It is saved in a csv file for you called "cruise_ship_info.csv". Your job is to create a regression model that will help predict how many crew members will be needed for future ships. The client also mentioned that they have found that particular cruise lines will differ in acceptable crew counts, so it is most likely an important feature to include in your analysis! 

Once you've created the model and tested it for a quick check on how well you can expect it to perform, make sure you take a look at why it performs so well!

In [1]:
import findspark
findspark.init('/home/gkouskosv/spark-2.4.5-bin-hadoop2.6/')

In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.appName('lr_cons_hyundai').getOrCreate()

In [4]:
data = spark.read.csv('cruise_ship_info.csv', header=True, inferSchema=True)

In [5]:
data.printSchema()

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)



In [6]:
data.columns

['Ship_name',
 'Cruise_line',
 'Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'crew']

In [7]:
data.show()

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|  Ship_name|Cruise_line|Age|           Tonnage|passengers|length|cabins|passenger_density|crew|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|    Journey|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|      Quest|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|Celebration|   Carnival| 26|            47.262|     14.86|  7.22|  7.43|             31.8| 6.7|
|   Conquest|   Carnival| 11|             110.0|     29.74|  9.53| 14.88|            36.99|19.1|
|    Destiny|   Carnival| 17|           101.353|     26.42|  8.92| 13.21|            38.36|10.0|
|    Ecstasy|   Carnival| 22|            70.367|     20.52|  8.55|  10.2|            34.29| 9.2|
|    Elation|   Carnival| 15|            70.367|     20.52|  8.55|  10.2|            34.29| 9.2|
|    Fantasy|   Carnival| 23| 

In [8]:
data.groupBy('Cruise_line').count().show()

+-----------------+-----+
|      Cruise_line|count|
+-----------------+-----+
|            Costa|   11|
|              P&O|    6|
|           Cunard|    3|
|Regent_Seven_Seas|    5|
|              MSC|    8|
|         Carnival|   22|
|          Crystal|    2|
|           Orient|    1|
|         Princess|   17|
|        Silversea|    4|
|         Seabourn|    3|
| Holland_American|   14|
|         Windstar|    3|
|           Disney|    2|
|        Norwegian|   13|
|          Oceania|    3|
|          Azamara|    2|
|        Celebrity|   10|
|             Star|    6|
|  Royal_Caribbean|   23|
+-----------------+-----+



## Setting Up DataFrame for Machine Learning 

In [9]:
# Import VectorAssembler, SrtingIndexer and Vectors
from pyspark.ml.regression import LinearRegression
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import StringIndexer, VectorAssembler

In [20]:
stringInexer = StringIndexer(inputCol='Cruise_line', outputCol='Cruise_labeled')

In [21]:
s = stringInexer.fit(data)

In [22]:
td = s.transform(data)

In [23]:
td.show()

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+--------------+
|  Ship_name|Cruise_line|Age|           Tonnage|passengers|length|cabins|passenger_density|crew|Cruise_labeled|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+--------------+
|    Journey|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|          16.0|
|      Quest|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|          16.0|
|Celebration|   Carnival| 26|            47.262|     14.86|  7.22|  7.43|             31.8| 6.7|           1.0|
|   Conquest|   Carnival| 11|             110.0|     29.74|  9.53| 14.88|            36.99|19.1|           1.0|
|    Destiny|   Carnival| 17|           101.353|     26.42|  8.92| 13.21|            38.36|10.0|           1.0|
|    Ecstasy|   Carnival| 22|            70.367|     20.52|  8.55|  10.2|            34.29| 9.2|        

In [57]:
td.select('Cruise_line', 'Cruise_labeled').distinct().orderBy(td['Cruise_labeled'].desc()).show()

+-----------------+--------------+
|      Cruise_line|Cruise_labeled|
+-----------------+--------------+
|           Orient|          19.0|
|          Crystal|          18.0|
|           Disney|          17.0|
|          Azamara|          16.0|
|           Cunard|          15.0|
|         Windstar|          14.0|
|         Seabourn|          13.0|
|          Oceania|          12.0|
|        Silversea|          11.0|
|Regent_Seven_Seas|          10.0|
|              P&O|           9.0|
|             Star|           8.0|
|              MSC|           7.0|
|        Celebrity|           6.0|
|            Costa|           5.0|
|        Norwegian|           4.0|
| Holland_American|           3.0|
|         Princess|           2.0|
|         Carnival|           1.0|
|  Royal_Caribbean|           0.0|
+-----------------+--------------+



In [58]:
# 2nd way with sorted() method
sorted(set([(i[0], i[1]) for i in td.select(['Cruise_line', 'Cruise_labeled']).collect()]), key=lambda x: x[1])

[('Royal_Caribbean', 0.0),
 ('Carnival', 1.0),
 ('Princess', 2.0),
 ('Holland_American', 3.0),
 ('Norwegian', 4.0),
 ('Costa', 5.0),
 ('Celebrity', 6.0),
 ('MSC', 7.0),
 ('Star', 8.0),
 ('P&O', 9.0),
 ('Regent_Seven_Seas', 10.0),
 ('Silversea', 11.0),
 ('Oceania', 12.0),
 ('Seabourn', 13.0),
 ('Windstar', 14.0),
 ('Cunard', 15.0),
 ('Azamara', 16.0),
 ('Disney', 17.0),
 ('Crystal', 18.0),
 ('Orient', 19.0)]

In [59]:
td.columns

['Ship_name',
 'Cruise_line',
 'Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'crew',
 'Cruise_labeled']

In [62]:
assembler = VectorAssembler(
    inputCols=['Age',
        'Tonnage',
        'passengers',
        'length',
        'cabins',
        'passenger_density',
        'Cruise_labeled'],
    outputCol='features')

In [63]:
output = assembler.transform(td)

In [69]:
output.select('features').show()

+--------------------+
|            features|
+--------------------+
|[6.0,30.276999999...|
|[6.0,30.276999999...|
|[26.0,47.262,14.8...|
|[11.0,110.0,29.74...|
|[17.0,101.353,26....|
|[22.0,70.367,20.5...|
|[15.0,70.367,20.5...|
|[23.0,70.367,20.5...|
|[19.0,70.367,20.5...|
|[6.0,110.23899999...|
|[10.0,110.0,29.74...|
|[28.0,46.052,14.5...|
|[18.0,70.367,20.5...|
|[17.0,70.367,20.5...|
|[11.0,86.0,21.24,...|
|[8.0,110.0,29.74,...|
|[9.0,88.5,21.24,9...|
|[15.0,70.367,20.5...|
|[12.0,88.5,21.24,...|
|[20.0,70.367,20.5...|
+--------------------+
only showing top 20 rows



In [71]:
 final_data = output.select(['features', 'crew'])

In [72]:
train_data, test_data = final_data.randomSplit([0.7,0.3])

In [73]:
train_data.describe().show()

+-------+------------------+
|summary|              crew|
+-------+------------------+
|  count|               100|
|   mean|7.7990000000000075|
| stddev| 3.657581996429616|
|    min|              0.59|
|    max|              21.0|
+-------+------------------+



### Create a Linear Regression Model object

In [ ]:
lr = 